In [ ]:
%matplotlib inline
import sys
import os

import pandas as pd
import numpy as np
import json
from datetime import datetime

import matplotlib.pyplot as plt

COLORS = [(0.12109375, 0.46484375, 0.703125),
          (0.99609375, 0.49609375, 0.0546875),
          (0.171875, 0.625, 0.171875),
          (0.8359375, 0.15234375, 0.15625),
          (0.578125, 0.40234375, 0.73828125),
          (0.546875, 0.3359375, 0.29296875),
          (0.88671875, 0.46484375, 0.7578125),
          (0.49609375, 0.49609375, 0.49609375),
          (0.734375, 0.73828125, 0.1328125),
          (0.08984375, 0.7421875, 0.80859375)]

import calendar

# Set font sizes
SMALL_SIZE = 16
MEDIUM_SIZE = 18
BIGGER_SIZE = 20
                
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [ ]:
def loadCAISO(year):
    dfp = pd.read_csv(os.path.join("../data/CAISO_DailyRenewablesWatch", 'DailyRenewablesWatch_%d.csv' % year),
                      index_col=0, parse_dates=True)
    dfp.index -= pd.Timedelta('7h')

    cols = [col for col in dfp.columns if col != 'carbon']
    dfp["total"] = dfp[cols].sum(axis=1)
    CARBON_INTENSITY = {"biogas":18, "biomass":18, "geo":42, "hydro":4,
                        "imports":428, "nuclear":16, "smhydro":4, "solarpv":46, "solarth":22,
                        "thermal":469, "wind":12}

    # recalculate carbon without the exports
    dfp["carbon"] = dfp.apply(lambda row:sum(row[fuel]*CARBON_INTENSITY[fuel]
                                           for fuel in CARBON_INTENSITY)/1e3, axis=1)
    dfp["carbon_intensity"] = dfp.apply(lambda row:row["carbon"]*1e3/row["total"], axis=1)
    return dfp

df = pd.concat([loadCAISO(y) for y in [2015,2016,2017,2018]])

df.dropna(inplace=True)
df.loc[:, 'year'] = df.index.year
df.loc[:, 'month'] = df.index.month
df.loc[:, 'hour'] = df.index.hour

In [ ]:
df_c = pd.read_csv("../data/curtailmentsMonthly.csv", index_col=0, parse_dates=True, usecols=[0, 1], names=["date", 'curtailment'], skiprows=1)
f, ax = plt.subplots()
ax.plot(df_c.curtailment/1000, marker='o')
ax.grid()
ax.set_xlabel("month")
ax.set_ylabel("GWh")

In [ ]:
df_monthly = df.loc[:, ["year", "month", "solarpv", "wind"]].groupby(["year", "month"]).sum().reset_index()
df_monthly.index = pd.to_datetime(df_monthly.year*10000+df_monthly.month*100+1,format='%Y%m%d')
df_monthly["gen"] = df_monthly.solarpv + df_monthly.wind

In [ ]:
f, ax = plt.subplots()
ax.plot(df_monthly.gen/1000, marker='o')
ax.plot(df_c.curtailment/1000, marker='o')
ax.grid()
ax.set_xlabel("month")
ax.set_ylabel("GWh")
f.autofmt_xdate()

In [ ]:
f, ax = plt.subplots()
ax.plot(df_c.curtailment/df_monthly.gen*100, marker='o')
ax.grid()
ax.set_xlabel("month")
ax.set_ylabel("Monthly curtailment (%)")
f.autofmt_xdate()

In [ ]:
df_c.curtailment/df_monthly.gen*100